In [4]:
pip install tensorflow

In [2]:
pip install keras

  Obtaining dependency information for keras from https://files.pythonhosted.org/packages/b0/b2/104733bb67fde86f3d10010f0b5c93cfa1d5bf552f904584cf9e5b3ba719/keras-3.0.5-py3-none-any.whl.metadata
  Obtaining dependency information for absl-py from https://files.pythonhosted.org/packages/a2/ad/e0d3c824784ff121c03cc031f944bc7e139a8f1870ffd2845cc2dd76f6c4/absl_py-2.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for rich from https://files.pythonhosted.org/packages/be/be/1520178fa01eabe014b16e72a952b9f900631142ccd03dc36cf93e30c1ce/rich-13.7.0-py3-none-any.whl.metadata
  Obtaining dependency information for namex from https://files.pythonhosted.org/packages/cd/43/b971880e2eb45c0bee2093710ae8044764a89afe9620df34a231c6f0ecd2/namex-0.0.7-py3-none-any.whl.metadata
  Obtaining dependency information for dm-tree from https://files.pythonhosted.org/packages/e4/c1/522041457444b67125ac9527208bb3148f63d7dce0a86ffa589ec763a10e/dm_tree-0.1.8-cp311-cp311-win_amd64.whl.metadata
  Obtaini

In [4]:
import pandas as pd
import numpy as np
import keras
import tensorflow
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense


data = pd.read_csv("train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head(12))

                                                 Text  Emotions
0   i can go from feeling so hopeless to so damned...   sadness
1    im grabbing a minute to post i feel greedy wrong     anger
2   i am ever feeling nostalgic about the fireplac...      love
3                                i am feeling grouchy     anger
4   ive been feeling a little burdened lately wasn...   sadness
5   ive been taking or milligrams or times recomme...  surprise
6   i feel as confused about life as a teenager or...      fear
7   i have been with petronas for years i feel tha...       joy
8                                 i feel romantic too      love
9   i feel like i have to make the suffering i m s...   sadness
10  i do feel that running is a divine experience ...       joy
11  i think it s the easiest time of year to feel ...     anger


In [5]:
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

In [6]:
sequences = tokenizer.texts_to_sequences(texts)
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length)

In [7]:
# Encode the string labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

In [8]:
# One-hot encode the labels
one_hot_labels = keras.utils.to_categorical(labels)

In [9]:
# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2)

In [11]:
# Define the model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, 
                    output_dim=128, input_length=max_length))
model.add(Flatten())
model.add(Dense(units=128, activation="relu"))
model.add(Dense(units=len(one_hot_labels[0]), activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(xtrain, ytrain, epochs=10, batch_size=32, validation_data=(xtest, ytest))



Epoch 1/10


400/400 [==============================] - 11s 25ms/step - loss: 1.3476 - accuracy: 0.4801 - val_loss: 0.8565 - val_accuracy: 0.7159
Epoch 2/10
400/400 [==============================] - 10s 24ms/step - loss: 0.3430 - accuracy: 0.8913 - val_loss: 0.5502 - val_accuracy: 0.8169
Epoch 3/10
400/400 [==============================] - 10s 24ms/step - loss: 0.0587 - accuracy: 0.9848 - val_loss: 0.5957 - val_accuracy: 0.8159
Epoch 4/10
400/400 [==============================] - 10s 24ms/step - loss: 0.0238 - accuracy: 0.9950 - val_loss: 0.6255 - val_accuracy: 0.8194
Epoch 5/10
400/400 [==============================] - 10s 25ms/step - loss: 0.0164 - accuracy: 0.9964 - val_loss: 0.6435 - val_accuracy: 0.8209
Epoch 6/10
400/400 [==============================] - 10s 24ms/step - loss: 0.0123 - accuracy: 0.9974 - val_loss: 0.6784 - val_accuracy: 0.8184
Epoch 7/10
400/400 [==============================] - 10s 24ms/step - loss: 0.0128 - accuracy: 0.9971 - val_loss: 0.7134 - val_accur

In [12]:
input_text = "She didn't come today because she lost her dog yestertay!"

# Preprocess the input text
input_sequence = tokenizer.texts_to_sequences([input_text])
padded_input_sequence = pad_sequences(input_sequence, maxlen=max_length)
prediction = model.predict(padded_input_sequence)
predicted_label = label_encoder.inverse_transform([np.argmax(prediction[0])])
print(predicted_label)

1/1 [==============================] - 0s 95ms/step
['sadness']
